# Combining & Merging datasets

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

Data contained in pandas objects can be combined together in a number of ways:

1. [**Database-Style DataFrame Joins**](#Database-Style-DataFrame-Joins)

    - [**Merge**](#Merge) [`pd.merge(df1, df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge): for combining data on common columns or indices

    - [**Join**](#Join) [`df1.join(df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join): for combining data on a key column or an index
    

2. [**Concatenating Along an Axis**](#Concatenating-Along-an-Axis)

    - [**Concat**](#Concat) [`pd.concat([df1, df2])`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat): for combining DataFrames across rows or columns

    - [**Append**](#Append) [`df1.append(df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html): stacking vertically

    ---

1. **Uniones de DataFrame de estilo de base de datos**:

- **Merge**: `pd.merge(df1, df2)` --> para combinar datos en columnas o índices comunes

- **Join**: `df1.join(df2)` --> para combinar datos en una columna clave o un índice

2. **Concatenación a lo largo de un eje**:

- **Concat**: `pd.concat([df1, df2])` --> para combinar DataFrames en filas o columnas

- **Append**: `df1.append(df2)` --> apilado verticalmente

------------------------------------------------------------------------------------------------------------------------------

## Database-Style DataFrame Joins

# Merge

Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on. When performing a cross merge, no column specifications to merge on are allowed.

---

 Fusionar(**merge**) objetos DataFrame o Series con nombre con una combinación de estilo de base de datos.

La unión se realiza en columnas o índices. Si une columnas en columnas, se ignorarán los índices de DataFrame. De lo contrario, si une índices en índices o índices en una columna o columnas, el índice se transferirá. Al realizar una fusión cruzada, no se permiten especificaciones de columna para fusionar.

When you use merge(), you’ll provide two required arguments:

- The left DataFrame
- The right DataFrame

After that, you can provide a number of optional arguments to define how your datasets are merged:

**how**: This defines what kind of merge to make. It defaults to 'inner', but other possible options include 'outer', 'left', and 'right'.

**on**: Use this to tell merge() which columns or indices (also called key columns or key indices) you want to join on. This is optional. If it isn’t specified, and left_index and right_index (covered below) are False, then columns from the two DataFrames that share names will be used as join keys. If you use on, then the column or index you specify must be present in both objects.

**left_on and right_on**: Use either of these to specify a column or index that is present only in the left or right objects that you are merging. Both default to None.

**left_index and right_index**: Set these to True to use the index of the left or right objects to be merged. Both default to False.

**suffixes**: This is a tuple of strings to append to identical column names that are not merge keys. This allows you to keep track of the origins of columns with the same name.

---


Cuando use **merge()**, proporcionará dos argumentos obligatorios:

- El DataFrame izquierdo
- El DataFrame derecho

Después de eso, puede proporcionar una serie de argumentos opcionales para definir cómo se fusionan sus conjuntos de datos:

**how**: Esto define qué tipo de fusión realizar. El valor predeterminado es 'inner', pero otras opciones posibles incluyen 'outer', 'left' y 'right'.

**on**: Use esto para decirle a merge () a qué columnas o índices (también llamados columnas clave o índices clave) desea unirse. Esto es opcional. Si no se especifica, y left_index y right_index (que se tratan a continuación) son falsos, las columnas de los dos DataFrames que comparten nombres se utilizarán como claves de combinación. Si usa on, la columna o índice que especifique debe estar presente en ambos objetos.

**left_on y right_on**: use cualquiera de estos para especificar una columna o índice que esté presente solo en los objetos de la izquierda o la derecha que está fusionando. Ambos valores predeterminados son None.

**left_index y right_index**: configúrelos en True para usar el índice de los objetos izquierdo o derecho que se fusionarán. Ambos valores predeterminados son Falso.

**suffixes**: esta es una tupla de cadenas para agregar a nombres de columna idénticos que no son claves de combinación. Esto le permite realizar un seguimiento de los orígenes de las columnas con el mismo nombre.

### Joining (Merging) DataFrames

Using the [MovieLens 100k data](http://grouplens.org/datasets/movielens/), let's create two DataFrames:

- **movies**: shows information about movies, namely a unique **movie_id** and its **title**
- **ratings**: shows the **rating** that a particular **user_id** gave to a particular **movie_id** at a particular **timestamp**

---

Con los datos de MovieLens 100k, creemos dos DataFrames:

- **movies**: muestra información sobre películas, a saber, un **movie_id** único y su **title**.
- **ratings**: muestra la **rating** que un **user_id** en particular le dio a un **movie_id** en particular en una **timestamp** en particular

In [59]:
#first things first
import pandas as pd
import numpy as np

#### Movies

In [60]:
url = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.item"

In [61]:
movie_cols = ['movie_id', 'title']
movies = pd.read_csv(url, sep='|', header=None, names=movie_cols, usecols=[0, 1])   # usecols selecciona las columnas que va a usar                                                                                         del dataset
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [62]:
movies.shape

(1682, 2)

In [63]:
movies.movie_id.nunique()

1682

#### Ratings

In [64]:
url1 = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.data"

In [65]:
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(url1, sep='\t', header=None, names=rating_cols)

In [66]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [67]:
ratings.shape

(100000, 4)

In [68]:
ratings.movie_id.nunique()

1682

In [69]:
ratings.loc[ratings.movie_id == 1, :]

,user_id,movie_id,rating,timestamp
24,308,1,4,887736532
454,287,1,5,875334088
957,148,1,4,877019411
971,280,1,4,891700426
1324,66,1,3,883601324
...,...,...,...,...
99423,895,1,4,879437950
99445,747,1,5,888639138
99631,786,1,4,882841828
99704,800,1,4,887646283


#### Merging Movies and Ratings

Let's pretend that you want to examine the ratings DataFrame, but you want to know the **title** of each movie rather than its **movie_id**. The best way to accomplish this objective is by "joining" (or "merging") the DataFrames using the Pandas `merge` function:

---

Supongamos que desea examinar los ratings DataFrame, pero desea saber el título de cada película en lugar de su **movie_id**. La mejor manera de lograr este objetivo es "joining" (o "merging") los DataFrames usando la `merge` de Pandas:

In [70]:
movies.shape

(1682, 2)

In [71]:
ratings.shape

(100000, 4)

In [72]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp'], dtype='object')

In [73]:
movies.columns

Index(['movie_id', 'title'], dtype='object')

In [74]:
movie_ratings = pd.merge(movies, ratings, how = 'outer') # OBSERVAR COMO LAS COLUMNAS REPETIDAS (movie_id en este caso) APARECEN UNA UNICA                                                                 VEZ
movie_ratings                 

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1,889289570
99996,1679,B. Monkey (1998),863,3,889289491
99997,1680,Sliding Doors (1998),863,2,889289570
99998,1681,You So Crazy (1994),896,3,887160722


In [75]:
prueba_how = pd.merge(movies, ratings,on='movie_id', how= 'inner')
prueba_how

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1,889289570
99996,1679,B. Monkey (1998),863,3,889289491
99997,1680,Sliding Doors (1998),863,2,889289570
99998,1681,You So Crazy (1994),896,3,887160722


In [76]:
movie_ratings

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1,889289570
99996,1679,B. Monkey (1998),863,3,889289491
99997,1680,Sliding Doors (1998),863,2,889289570
99998,1681,You So Crazy (1994),896,3,887160722


In [77]:
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [78]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [79]:
movie_ratings.head()

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324


In [80]:
movie_ratings.shape

(100000, 5)

In [81]:
movie_ratings[movie_ratings['movie_id'] == 1].count()

movie_id     452
title        452
user_id      452
rating       452
timestamp    452
dtype: int64

In [82]:
movie_ratings['movie_id'][movie_ratings['movie_id'] == 1].count()


452

Here's what just happened:

- Pandas noticed that movies and ratings had one column in common, namely **movie_id**. This is the "key" on which the DataFrames will be joined.
- The first **movie_id** in movies is 1. Thus, Pandas looked through every row in the ratings DataFrame, searching for a movie_id of 1. Every time it found such a row, it recorded the **user_id**, **rating**, and **timestamp** listed in that row. In this case, it found 452 matching rows.
- The second **movie_id** in movies is 2. Again, Pandas did a search of ratings and found 131 matching rows.
- This process was repeated for all of the remaining rows in movies.

At the end of the process, the movie_ratings DataFrame is created, which contains the two columns from movies (**movie_id** and **title**) and the three other colums from ratings (**user_id**, **rating**, and **timestamp**).

- **movie_id** 1 and its **title** are listed 452 times, next to the **user_id**, **rating**, and **timestamp** for each of the 452 matching ratings.
- **movie_id** 2 and its **title** are listed 131 times, next to the **user_id**, **rating**, and **timestamp** for each of the 131 matching ratings.
- And so on, for every movie in the dataset.

---

Esto es lo que acaba de pasar:

- Pandas notó que las películas y las calificaciones tenían una columna en común, **movie_id**. Esta es la "KEY" en la que se unirán los DataFrames.
- El primer **movie_id** en las películas es 1. Por lo tanto, Pandas examinó todas las filas del DataFrame de calificaciones, buscando un movie_id de 1. Cada vez que encontraba una fila de este tipo, registraba el **user_id**, la **rating** y **timestamp** enumerados en esa fila. En este caso, encontró 452 filas coincidentes.
- El segundo **movie_id** en las películas es 2. Nuevamente, Pandas hizo una búsqueda de calificaciones y encontró 131 filas coincidentes.
- Este proceso se repitió para todas las filas restantes de las películas.

Al final del proceso, se crea el marco de datos movie_ratings, que contiene las dos columnas de películas (**movie_id** y **title**) y las otras tres columnas de calificaciones (**user_id**, **rating** y **timestamp**).

- **movie_id** 1 y su **title** se enumeran 452 veces, junto al **user_id**, **rating** y **timestamp** para cada una de las 452 calificaciones coincidentes.
- **movie_id** 2 y su **title** se enumeran 131 veces, junto al **user_id**, **rating** y **timestamp** para cada una de las 131 ratings coincidentes.
- Y así sucesivamente, para cada película del conjunto de datos.

In [83]:
movie_ratings.loc[movie_ratings.movie_id == 1, :]

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
447,1,Toy Story (1995),895,4,879437950
448,1,Toy Story (1995),747,5,888639138
449,1,Toy Story (1995),786,4,882841828
450,1,Toy Story (1995),800,4,887646283


In [84]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

(1682, 2)
(100000, 4)
(100000, 5)


Notice the shapes of the three DataFrames:

- There are 1682 rows in the movies DataFrame.
- There are 100000 rows in the ratings DataFrame.
- The `merge` function resulted in a movie_ratings DataFrame with 100000 rows, because every row from ratings matched a row from movies.
- The movie_ratings DataFrame has 5 columns, namely the 2 columns from movies, plus the 4 columns from ratings, minus the 1 column in common.

By default, the `merge` function joins the DataFrames using all column names that are in common (**movie_id**, in this case). The [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge.html) explains how you can override this behavior.

---

Observe las formas de los tres DataFrames:

- Hay 1682 filas en el DataFrame de películas.
- Hay 100000 filas en el ratings DataFrame
- La función `merge` dio como resultado un DataFrame de movie_ratings con 100000 filas, porque cada fila de las calificaciones coincidía con una fila de las películas.
- El DataFrame de movie_ratings tiene 5 columnas, a saber, las 2 columnas de películas, más las 4 columnas de calificaciones, menos la columna en común(**movie_id**).

De forma predeterminada, la función `merge` une los DataFrames utilizando todos los nombres de columna que son comunes (**movie_id**, en este caso). La [documentación](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge.html) explica cómo puede anular este comportamiento.

**What if the columns you want to join on don't have the same name?**

In [85]:
movies.columns = ['m_id', 'title']

In [86]:
movies.columns

Index(['m_id', 'title'], dtype='object')

In [87]:
movies

,m_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [88]:
pd.merge(movies, ratings, left_on = 'm_id', right_on = 'movie_id')

,m_id,title,user_id,movie_id,rating,timestamp
0,1,Toy Story (1995),308,1,4,887736532
1,1,Toy Story (1995),287,1,5,875334088
2,1,Toy Story (1995),148,1,4,877019411
3,1,Toy Story (1995),280,1,4,891700426
4,1,Toy Story (1995),66,1,3,883601324
...,...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1678,1,889289570
99996,1679,B. Monkey (1998),863,1679,3,889289491
99997,1680,Sliding Doors (1998),863,1680,2,889289570
99998,1681,You So Crazy (1994),896,1681,3,887160722


#### What if you want to join on one index?

In [89]:
ratings_index = ratings.set_index('movie_id')
ratings_index

,user_id,rating,timestamp
movie_id,,,
242,196,3,881250949
302,186,3,891717742
377,22,1,878887116
51,244,2,880606923
346,166,1,886397596
...,...,...,...
476,880,3,880175444
204,716,5,879795543
1090,276,1,874795795


In [90]:
pd.merge(movies, ratings_index, left_on = 'm_id', right_index = True)

,m_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
0,1,Toy Story (1995),287,5,875334088
0,1,Toy Story (1995),148,4,877019411
0,1,Toy Story (1995),280,4,891700426
0,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
1677,1678,Mat' i syn (1997),863,1,889289570
1678,1679,B. Monkey (1998),863,3,889289491
1679,1680,Sliding Doors (1998),863,2,889289570
1680,1681,You So Crazy (1994),896,3,887160722


#### What if you want to join on two indexes?

In [91]:
movies_index = movies.set_index('m_id')
movies_index.head()

,title
m_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [92]:
ratings_index.head()

,user_id,rating,timestamp
movie_id,,,
242,196,3,881250949
302,186,3,891717742
377,22,1,878887116
51,244,2,880606923
346,166,1,886397596


In [93]:
pd.merge(movies_index, ratings_index, left_index = True, right_index = True)

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


### Types of join

There are actually four types of joins supported by the Pandas `merge` function. Here's how they are described by the documentation:

- **inner:** use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys
- **outer:** use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
- **left:** use only keys from left frame, similar to a SQL left outer join; preserve key order
- **right:** use only keys from right frame, similar to a SQL right outer join; preserve key order

The default is the "inner join", which was used when creating the movie_ratings DataFrame.

It's easiest to understand the different types by looking at some simple examples:

---


En realidad, hay cuatro tipos de combinaciones compatibles con la función `merge` de Pandas. Así es como se describen en la documentación:

- **inner:**  use la intersección de claves de ambos marcos, similar a una combinación interna de SQL; preservar el orden de las keys de la izquierda
- **outer:** use la unión de claves de ambos marcos, similar a una unión externa completa de SQL; ordenar claves lexicográficamente
- **left:** use solo las claves del marco izquierdo, similar a una combinación externa izquierda de SQL; preservar el orden de las claves
- **right:** use solo las claves del marco derecho, similar a una combinación externa derecha de SQL; preservar el orden de las claves

El valor predeterminado es la "combinación interna", que se utilizó al crear el marco de datos movie_ratings.

Es más fácil comprender los diferentes tipos observando algunos ejemplos simples:

![join](https://www.dofactory.com/img/sql/sql-joins.png)

In [94]:
A = pd.DataFrame({'color': ['green', 'yellow', 'red'], 'num':[1, 2, 3]})
A

,color,num
0,green,1
1,yellow,2
2,red,3


In [95]:
B = pd.DataFrame({'color': ['green', 'yellow', 'pink'], 'size':['S', 'M', 'L']})
B

,color,size
0,green,S
1,yellow,M
2,pink,L


#### Inner join

Only include observations found in both A and B:

In [96]:
pd.merge(A, B, how = 'inner')  # VER QUE LOS VALORES PARA RED Y PINK DESAPARECEN PORQUE NO ESTAN EN AMBOS DF (no hay interseccion)

,color,num,size
0,green,1,S
1,yellow,2,M


In [97]:
pd.merge(A, B, left_index = True, right_index = True, how = 'inner')  # Los SUFIJOS predeterminados son "_x" y "_y"

,color_x,num,color_y,size
0,green,1,green,S
1,yellow,2,yellow,M
2,red,3,pink,L


#### Outer join

Include observations found in either A or B:

In [98]:
pd.merge(A, B, how = 'outer')

,color,num,size
0,green,1.0,S
1,yellow,2.0,M
2,red,3.0,NaN
3,pink,NaN,L


#### Left join

Include all observations found in A:

In [99]:
pd.merge(A, B, how = 'left')

,color,num,size
0,green,1,S
1,yellow,2,M
2,red,3,NaN


#### Right join

Include all observations found in B:

In [100]:
pd.merge(A, B, how = 'right')

,color,num,size
0,green,1.0,S
1,yellow,2.0,M
2,pink,NaN,L


In [101]:
pd.merge(A, B, left_index = True, right_index = True, how = 'right')

,color_x,num,color_y,size
0,green,1,green,S
1,yellow,2,yellow,M
2,red,3,pink,L


In [102]:
pd.merge(A, B, left_index = True, right_index = True, how = 'left')

,color_x,num,color_y,size
0,green,1,green,S
1,yellow,2,yellow,M
2,red,3,pink,L


In [103]:
# con el ejemplo de movies index y ratings index
pd.merge(movies_index, ratings_index, left_index = True, right_index = True, how = "outer")

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


# Join

Join columns of another DataFrame.

Join columns with other DataFrame either on index or on a key column. Efficiently join multiple DataFrame objects by index at once by passing a list.

If we want to join using the key columns, we need to set key to be the index in both df and other. The joined DataFrame will have key as its index.

While `merge` is a module function, `.join` is an object function that lives on your DataFrame. This enables you to specify only one DataFrame, which will join the DataFrame you call `.join` on.

Under the hood, `.join` uses `merge`

---

Une columnas de otro DataFrame.

Unir columnas con otro DataFrame en el índice o en una columna clave. Une eficientemente varios objetos DataFrame por índice a la vez pasando una lista.

Si queremos unirnos usando las columnas clave, necesitamos configurar la clave para que sea el índice tanto en df como en otros. El DataFrame unido tendrá key como índice.

Si bien `merge` es una función de módulo, `.join` es una función de objeto que vive en su DataFrame. Esto le permite especificar solo un DataFrame, que se unirá al DataFrame al que llama `.join`.

Debajo del capó, `.join` usa `merge`

In [104]:
B.join(A, lsuffix = "_B", rsuffix = "_A")

,color_B,size,color_A,num
0,green,S,green,1
1,yellow,M,yellow,2
2,pink,L,red,3


In [125]:
A.join(B)

ValueError: columns overlap but no suffix specified: Index(['color'], dtype='object')

In [123]:
movies.join(ratings) # ACA LAS COLUMNAS NO SE SUPERPONEN PORQUE ANTERIORMENTE SE LE CAMBIO EL NOMBRE A LA COLUMNA movie_id del DF "movies" por "m_id".

,m_id,title,user_id,movie_id,rating,timestamp
0,1,Toy Story (1995),196,242,3,881250949
1,2,GoldenEye (1995),186,302,3,891717742
2,3,Four Rooms (1995),22,377,1,878887116
3,4,Get Shorty (1995),244,51,2,880606923
4,5,Copycat (1995),166,346,1,886397596
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),303,94,3,879485318
1678,1679,B. Monkey (1998),99,50,5,885679998
1679,1680,Sliding Doors (1998),306,14,5,876503995
1680,1681,You So Crazy (1994),92,709,2,875654590


In [105]:
#con las películas
display(movies.head())
ratings.head()

,m_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [106]:
movies.join(ratings)

,m_id,title,user_id,movie_id,rating,timestamp
0,1,Toy Story (1995),196,242,3,881250949
1,2,GoldenEye (1995),186,302,3,891717742
2,3,Four Rooms (1995),22,377,1,878887116
3,4,Get Shorty (1995),244,51,2,880606923
4,5,Copycat (1995),166,346,1,886397596
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),303,94,3,879485318
1678,1679,B. Monkey (1998),99,50,5,885679998
1679,1680,Sliding Doors (1998),306,14,5,876503995
1680,1681,You So Crazy (1994),92,709,2,875654590


In [107]:
ratings.iloc[[0]]

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949


In [108]:
movies.iloc[[0]]

,m_id,title
0,1,Toy Story (1995)


In [109]:
ratings.join(movies)

,user_id,movie_id,rating,timestamp,m_id,title
0,196,242,3,881250949,1.0,Toy Story (1995)
1,186,302,3,891717742,2.0,GoldenEye (1995)
2,22,377,1,878887116,3.0,Four Rooms (1995)
3,244,51,2,880606923,4.0,Get Shorty (1995)
4,166,346,1,886397596,5.0,Copycat (1995)
...,...,...,...,...,...,...
99995,880,476,3,880175444,NaN,NaN
99996,716,204,5,879795543,NaN,NaN
99997,276,1090,1,874795795,NaN,NaN
99998,13,225,2,882399156,NaN,NaN


In [118]:
display(ratings_index.head(), movies_index.head())

,user_id,rating,timestamp
movie_id,,,
242,196,3,881250949
302,186,3,891717742
377,22,1,878887116
51,244,2,880606923
346,166,1,886397596


,title
m_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [76]:
#si tienen mismo index, igual que al hacer pd.merge(movies_index, ratings_index, left_index=True, right_index=True)
movies_index.join(ratings_index)

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


In [140]:
pd.merge(movies_index, ratings_index, left_index=True, right_index= True)

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


# Concatenating Along an Axis

# Concat

Concatenate pandas objects along a particular axis with optional set logic along the other axes.

Can also add a layer of hierarchical indexing on the concatenation axis, which may be useful if the labels are the same (or overlapping) on the passed axis number.

---

Concatenar objetos pandas a lo largo de un eje particular con una lógica de conjunto opcional a lo largo de los otros ejes.

También puede agregar una capa de indexación jerárquica en el eje de concatenación, lo que puede ser útil si las etiquetas son las mismas (o se superponen) en el número de eje pasado.

Concatenation is a bit different from the merging techniques you saw above. With merging, you can expect the resulting dataset to have rows from the parent datasets mixed in together, often based on some commonality. Depending on the type of merge, you might also lose rows that don’t have matches in the other dataset.

With concatenation, your datasets are just stitched together along an axis — either the row axis or column axis. Visually, a concatenation with no parameters along rows would look like this:

---

La concatenación es un poco diferente de las técnicas de fusión que vio anteriormente. Con la fusión, puede esperar que el conjunto de datos resultante tenga filas de los conjuntos de datos principales mezclados, a menudo en función de algunos puntos en común. Según el tipo de combinación, es posible que también pierda filas que no tengan coincidencias en el otro conjunto de datos.

Con la concatenación, sus conjuntos de datos simplemente se unen a lo largo de un eje, ya sea el eje de fila o el eje de columna. Visualmente, una concatenación sin parámetros a lo largo de las filas se vería así:

![](https://files.realpython.com/media/concat_axis0.2ec65b5f72bc.png)

To implement this in code, you’ll use concat() and pass it a list of DataFrames that you want to concatenate.

---

Para implementar esto en el código, usará concat () y le pasará una lista de DataFrames que desea concatenar.

**objs**: This parameter takes any sequence (typically a list) of Series or DataFrame objects to be concatenated. You can also provide a dictionary. In this case, the keys will be used for the keys options.

**axis**: Like in the other techniques, this represents the axis you will concatenate along. The default value is 0, which concatenates along the index (or row axis), while 1 concatenates along columns (vertically). You can also use the string values index or columns.

**join**: This is similar to the how parameter in the other techniques, but it only accepts the values inner or outer. The default value is outer, which preserves data, while inner would eliminate data that does not have a match in the other dataset.

---

**objs**: este parámetro toma cualquier secuencia (normalmente una lista) de objetos Series o DataFrame para concatenar. También puede proporcionar un diccionario. En este caso, las claves se utilizarán para las opciones de claves.

**axis**: como en las otras técnicas, representa el eje a lo largo del cual concatenarás. El valor predeterminado es 0, que se concatena a lo largo del índice (o eje de fila), mientras que 1 se concatena a lo largo de columnas (verticalmente). También puede utilizar el índice o las columnas de valores de cadena.

**join**: Es similar al parámetro how de las otras técnicas, pero solo acepta los valores interno o externo. El valor predeterminado es externo, que conserva los datos, mientras que interno eliminaría los datos que no coinciden en el otro conjunto de datos.

In [141]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ["a", "b", "c"], columns = ["one", "three"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index = ["a", "c"], columns = ["three", "four"])

In [142]:
df1

,one,three
a,0,1
b,2,3
c,4,5


In [143]:
df2

,three,four
a,5,6
c,7,8


In [145]:
pd.concat([df1, df2], axis = 1, join='outer') # POR DEFECTO: axis = 0 - join = outer

,one,three,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [146]:
pd.concat([df1, df2], axis = 1, join='inner') # POR DEFECTO: axis = 0 - join = outer

,one,three,three,four
a,0,1,5,6
c,4,5,7,8


In [91]:
pd.concat([df1, df2], join = 'outer') # POR DEFECTO: axis = 0 - join = outer

,one,three,four
a,0.0,1,NaN
b,2.0,3,NaN
c,4.0,5,NaN
a,NaN,5,6.0
c,NaN,7,8.0


In [92]:
pd.concat([df1, df2], join = 'inner') # POR DEFECTO: axis = 0 - join = outer

,three
a,1
b,3
c,5
a,5
c,7


A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the `keys` argument.

---

Un problema potencial es que las piezas concatenadas no son identificables en el resultado. En su lugar, suponga que desea crear un índice jerárquico en el eje de concatenación. Para hacer esto, use el argumento `keys`.

In [80]:
pd.concat([df1, df2], axis = 1, keys = ["level1", "level2"])

level1       level2     
     one three  three four
a      0     1    5.0  6.0
b      2     3    NaN  NaN
c      4     5    7.0  8.0

If you pass a dict of objects instead of a list, the dict's keys will be used for the keys option: 

---

Si pasa un diccionario de objetos en lugar de una lista, las claves del dictado se usarán para la opción de claves:


In [94]:
pd.concat({"level1": df1, "level2": df2}, axis = 1)

level1       level2     
     one three  three four
a      0     1    5.0  6.0
b      2     3    NaN  NaN
c      4     5    7.0  8.0

In [149]:
#usando movies y ratings vemos que para este problema concat no sirve
pd.concat([movies, ratings], axis = 1, keys = ["movies", "ratings"], join = "inner")

movies                                            ratings           \
       m_id                                      title user_id movie_id   
0         1                           Toy Story (1995)     196      242   
1         2                           GoldenEye (1995)     186      302   
2         3                          Four Rooms (1995)      22      377   
3         4                          Get Shorty (1995)     244       51   
4         5                             Copycat (1995)     166      346   
...     ...                                        ...     ...      ...   
1677   1678                          Mat' i syn (1997)     303       94   
1678   1679                           B. Monkey (1998)      99       50   
1679   1680                       Sliding Doors (1998)     306       14   
1680   1681                        You So Crazy (1994)      92      709   
1681   1682  Scream of Stone (Schrei aus Stein) (1991)     227      295   

                        
     rating  timestamp  
0         3  881250949  
1         3  891717742  
2         1  878887116  
3         2  880606923  
4         1  886397596  
...     ...        ...  
1677      3  879485318  
1678      5  885679998  
1679      5  876503995  
1680      2  875654590  
1681      5  879035387  

[1682 rows x 6 columns]

# Append

Because direct array concatenation is so common, ``Series`` and ``DataFrame`` objects have an ``append`` method that can accomplish the same thing in fewer keystrokes.

Append is the specific case `(axis=0, join='outer')` of concat

For example, rather than calling ``pd.concat([df1, df2])``, you can simply call ``df1.append(df2)``:

---

Debido a que la concatenación directa de matrices es tan común, los objetos ``Series`` y ``DataFrame`` tienen un método ``append`` que puede lograr lo mismo con menos pulsaciones de teclas.

Append es el caso específico `(axis = 0, join = 'outer')` de concat

Por ejemplo, en lugar de llamar a ``pd.concat([df1, df2])``, simplemente puede llamar a ``df1.append(df2)``:

In [83]:
df_a = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df2_a = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))

In [100]:
df_a

,A,B
0,1,2
1,3,4


In [101]:
df2_a

,A,B
0,5,6
1,7,8


In [102]:
df_a.append(df2_a)

,A,B
0,1,2
1,3,4
0,5,6
1,7,8


In [103]:
%timeit df_a.append(df2_a)

225 µs ± 5.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [104]:
%timeit pd.concat([df_a, df2_a])

218 µs ± 5.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [105]:
# concat también tiene el parámetro ignore_index = True
df_a.append(df2_a, ignore_index = True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


In [90]:
# esto sería lo mismo que usar ignore_index
df_a.append(df2_a).reset_index(drop=True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8
